In [35]:
import scipy
import json
import keras_cv
import keras
import matplotlib.pyplot as plt
from keras import ops
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import MDS, Isomap, LocallyLinearEmbedding
# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
prompt_1 = "A watercolor painting of a Golden Retriever at the beach"
prompt_2 = "A still life DSLR photo of a bowl of fruit"
interpolation_steps = 5

encoding_1 = ops.squeeze(model.encode_text(prompt_1))
encoding_2 = ops.squeeze(model.encode_text(prompt_2))

interpolated_encodings = ops.linspace(encoding_1, encoding_2, interpolation_steps)

# Show the size of the latent manifold
print(f"Encoding shape: {encoding_1.shape}")


Encoding shape: (77, 768)


In [61]:
def word_similarity(word1: str, word2: str, encoding_model) -> float:
    encoding1 = encoding_model.encode_text(word1)
    encoding2 = encoding_model.encode_text(word2)
    distance = scipy.spatial.distance.euclidean(encoding1.flatten(), encoding2.flatten())

    return distance

In [62]:
word1 = "king"
word2 = "corolla"
print(word_similarity(word1, word2, model))

248.2


In [54]:
print(model.encode_text(word2).shape)

(1, 77, 768)


# Dimensionality Reduction

In [38]:
with open("data/connections.json") as file:
        connections = json.load(file)

In [58]:
X_train = np.empty((16*(len(connections)), 77, 768))
print(X_train.shape)

(6032, 77, 768)


In [60]:
i = 0
for puzzle in connections:
    for level in puzzle["answers"]:
        for word in level["members"]:
            X_train[i] = model.encode_text(word)
            i += 1

In [31]:
lda = Pipeline([('scaler', StandardScaler()),
               ('lda', LDA(n_components=2))])

lda.fit(X_train, t_train)
y_train_lda = lda.transform(X_train)

In [ ]:
mds = MDS(n_components=2, dissimilarity='euclidean', normalized_stress='auto')

# new coordinates
Y = mds.fit_transform(X_train)

In [ ]:
iso = Isomap(n_components=2)
proj_isomap = iso.fit_transform(X_train)

In [ ]:
lle = LocallyLinearEmbedding(n_components=2)
proj_lle = lle.fit_transform(X_train)